In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


Chamando as bases

In [35]:
df18 = pd.read_excel('ESTADIC//Base_ESTADIC_2018.xlsx', sheet_name='Educação', usecols=['Cod Uf','EEDU161A','EEDU161B'])
df18

,Cod Uf,EEDU161A,EEDU161B
0,11,680,2012
1,12,67,1999
2,13,3951,2013
3,14,892,2013
4,15,7442,2010
5,16,949,2005
6,17,2859,2014
7,21,9860,2013
8,22,71,2006
9,23,12066,1993


In [36]:
df21 = pd.read_excel('ESTADIC//Base_ESTADIC_2021.xlsx', sheet_name='Educação', usecols=['Cod UF','EEDU161A','EEDU161B'])
df21

,Cod UF,EEDU161A,EEDU161B
0,11,680,2016
1,12,330,2017
2,13,3951,2013
3,14,892,2014
4,15,7442,2010
5,16,949,2015
6,17,2851,2014
7,21,9860,2013
8,22,71,2006
9,23,15575,2014


In [37]:
uf = pd.read_excel('ESTADIC//Base_ESTADIC_2018.xlsx', sheet_name = 'Variáveis externas', usecols=[1,2,3])
uf

,COD UF,UF,NOME UF
0,11,RO,RONDÔNIA
1,12,AC,ACRE
2,13,AM,AMAZONAS
3,14,RR,RORAIMA
4,15,PA,PARÁ
5,16,AP,AMAPÁ
6,17,TO,TOCANTINS
7,21,MA,MARANHÃO
8,22,PI,PIAUÍ
9,23,CE,CEARÁ


Renomeando as colunas

In [38]:
df21.columns

Index(['Cod UF', 'EEDU161A', 'EEDU161B'], dtype='object')

In [39]:
df18= df18.rename(columns={'Cod Uf':'cod_uf',
                        'EEDU161A':'lei_criacao_plano_carreira',
                        'EEDU161B':'ano_criacao_lei'}) 

In [40]:
df21= df21.rename(columns={'Cod UF':'cod_uf',
                        'EEDU161A':'lei_criacao_plano_carreira',
                        'EEDU161B':'ano_criacao_lei'}) 

In [41]:
##adicionando a coluna de 'ano' em cada um dos dfs
df18['ano']=2018
df21['ano']=2021 

In [42]:
df = pd.concat([df18,df21]) #juntando os dataframes

In [43]:
x= uf.pivot_table(columns=('COD UF', 'UF', 'NOME UF'), aggfunc='size')


In [44]:
uf = pd.DataFrame(x).reset_index()[['COD UF', 'UF', 'NOME UF']]

In [45]:
df = df.merge(uf, right_on='COD UF',left_on='cod_uf') #adicionando sigla e nome das UFs


In [46]:
df = df.drop(['COD UF'], axis=1) #eliminando coluna repetida

In [47]:
df= df.rename(columns={'UF':'sigla_uf',
                'NOME UF':'uf'})

In [48]:
df['uf'] = df['uf'].str.title()

In [52]:
df = df[['ano','cod_uf','sigla_uf','uf','ano_criacao_lei','lei_criacao_plano_carreira']] #organizando as colunas

Subindo para o GBQ

In [54]:
client = bigquery.Client()
dataset_ref = client.dataset('estrutura_organizacional_carreiras')

In [55]:
df.dtypes

ano                            int64
cod_uf                         int64
sigla_uf                      object
uf                            object
ano_criacao_lei                int64
lei_criacao_plano_carreira     int64
dtype: object

In [58]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
        bigquery.SchemaField('cod_uf','INTEGER',description='Código do IBGE da UF'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da UF'),
        bigquery.SchemaField('uf','STRING',description='Nome da UF'),        
        bigquery.SchemaField('ano_criacao_lei','INTEGER',description='Se a nomeação de diretor de escola ocorreu por concurso público '),
        bigquery.SchemaField('lei_criacao_plano_carreira','INTEGER',description='Se a nomeação de diretor de escola ocorreu por indicação')
        ]


In [59]:
table_ref = dataset_ref.table('ESTADIC_lei_plano_carreira_por_ano')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=0c42721b-8541-4039-a059-12db189b10eb>